In [1]:
import pandas as pd
from babel import Locale
from babel import core
import pycountry
from difflib import SequenceMatcher
from prompter import yesno
import operator
from math import*

In [2]:
# unesco demo data
unesco_demo = pd.read_table("Data/unesco_demo_norm.csv", sep = "\t")
unesco_demo.head()

,Indicator,iso3,Year,Value,Flag
0,SP_DYN_TFRT_IN,AUS,2003,1.748,NaN
1,SP_DYN_TFRT_IN,AUS,2004,1.768,NaN
2,SP_DYN_TFRT_IN,AUS,2005,1.807,NaN
3,SP_DYN_TFRT_IN,AUS,2006,1.908,NaN
4,SP_DYN_TFRT_IN,AUS,2007,1.959,NaN


In [3]:
# unesco education data
unesco_education = pd.read_table("Data/unesco_education_norm.csv", sep = "\t")
unesco_education.head()

,Indicator,iso3,Year,Value,Flag
0,XGOVEXP_IMF,BFA,2005,19.49419,NaN
1,XGOVEXP_IMF,BFA,2006,18.20170,NaN
2,XGOVEXP_IMF,BFA,2007,17.86382,NaN
3,XGOVEXP_IMF,BFA,2010,17.25081,NaN
4,XGOVEXP_IMF,BFA,2011,20.31364,NaN


In [4]:
# country specific table
geo_info = pd.read_table("Data/cepii_geo_info_norm.csv", sep = "\t")
geo_info.head()

,iso3,country,area,dis_int,landlocked,continent,maincity,citynum,langoff_1,langoff_2,...,lang9_2,lang9_3,lang9_4,colonizer1,colonizer2,colonizer3,colonizer4,short_colonizer1,short_colonizer2,short_colonizer3
0,ABW,Aruba,193,5.225315,0,America,1,2,Dutch,Spanish,...,.,.,.,NLD,.,.,.,.,.,.
1,AFG,Afghanistan,652225,303.761400,1,Asia,1,25,Persian,.,...,Uzbek,.,.,.,.,.,.,GBR,.,.
2,AGO,Angola,1246700,419.966600,0,Africa,1,25,Portuguese,.,...,.,.,.,PRT,.,.,.,.,.,.
3,AIA,Anguilla,102,3.798690,0,America,1,2,English,.,...,.,.,.,GBR,.,.,.,.,.,.
4,ALB,Albania,28748,63.773110,0,Europe,1,25,Albanian,.,...,.,.,.,TUR,.,.,.,.,.,.


In [5]:
# pair of countries distances table
geo_dist = pd.read_table("Data/cepii_geo_dist.tsv", sep = "\t", na_values = {"distw": '.', "distwces": '.'})
geo_dist.head()

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
0,ABW,ABW,0,0,0,0,0,0,0,0,"5,225315","5,225315","25,09354","23,04723"
1,ABW,AFG,0,0,0,0,0,0,0,0,"13257,81","13257,81","13168,22","13166,37"
2,ABW,AGO,0,0,0,0,0,0,0,0,"9516,913","9516,913","9587,316","9584,193"
3,ABW,AIA,0,0,1,0,0,0,0,0,"983,2682","983,2682","976,8974","976,8916"
4,ABW,ALB,0,0,0,0,0,0,0,0,"9091,742","9091,742","9091,576","9091,466"


In [6]:
# main info table
resident_foreigners_norm = pd.read_table("Data/resident_foreigners_norm.csv", sep = "\t")
resident_foreigners_norm.head()

,Province,Country,Year,Gender,Value
0,Biella,Andorra,2005,male,1
1,Biella,Andorra,2005,female,0
2,Novara,Andorra,2005,male,0
3,Novara,Andorra,2005,female,2
4,Alessandria,Andorra,2017,male,0


In [7]:
# all countries (iso3 code) in the social_finance table
iso3_unesco = list(set(unesco_demo["iso3"]))
print(len(iso3_unesco))
# all countries (iso3 code) in the geo_info table
iso3_cepii = list(set(geo_info["iso3"]))
print(len(iso3_cepii))
# all countries (iso3 code) in both the tables
iso3 = list(set(iso3_unesco).union(iso3_cepii))
print(len(iso3))

227
225
243


In [8]:
# dictionary with the structure: {iso3: country} 
iso3_all_countrie_engl = {}
not_found = []
for i in iso3:
    try:
        iso3_all_countrie_engl[i] = pycountry.countries.get(alpha_3=i).name
    except:
        not_found.append(i)

In [9]:
not_found

['ROM', 'PAL', 'ANT', 'ZZA', 'TMP', 'ZAR', 'YUG']

The iso3 not found by the package are:
- ANT, from Cepii;
- PAl, from Cepii;
- ROM, from Cepii;
- TMP, from Cepii;
- YUM, from Cepii;
- ZAR, from Cepii;
- ZZA, from Unesco;

Why?
- ANT: for Cepii in Netherland Antilles.. there is Antille Olandesi in ISTAT, but only 1 person in 2015 coiming to Roma and Brindisi. Drop the information from Cepii and ISTAT tables;
- ZAR: for Cepii is Congo.. it's wrong! The right iso3 for Congo is COD.. right in Unesco. Rename the information in the Cepii table;
- ROM: for Cepii is Romania.. it's wrong! The right iso3 for Romania is ROU.. right in Unesco; Rename the information in the Cepii table;
- YUG: for Cepii is Serbia and Montenegro.. Unesco has separate fields Serbia - Montenegro.. consider the same distance for the two states;
- PAL: for Cepii in Palestina.. it's not actually a state.. there is no Palestina in ISTAT or Unesco. Drop the information from the Cepii table;
- TMP: for Cepii is Timor est (Timor-Lest).. it's wrong! The right iso3 for Timor-Lest is TLS.. right in Unesco; Rename the information in the Cepii table;
- ZZA: for Unesco is Channel Islands... there is no Channel Islands in ISTAT or Cepii. Drop the information from the Unesco table. 

Let's do all the needed update.. 

RECAL:
- all the update on Cepii have to be done both on cepii_geo_info and cepii_geo_dist!
- all the update on Unesco have to be done both on demo and education!

In [10]:
# Necessary update for the dataset in order to have coherence and no mistakes
# drop Netherland Antilles data 
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Country"] != "Antille Olandesi"]
geo_info = geo_info[geo_info["iso3"] != "ANT"]
geo_dist = geo_dist[(geo_dist["iso_d"] != "ANT") & (geo_dist["iso_o"] != "ANT")]

# change the iso3 and iso2 code of "Romania", "Congo (Democratic Republic of the)", "East Timor" - geo_info and geo_dist db
geo_info = geo_info.replace({"ROM": "ROU", "RO": pycountry.countries.get(alpha_3="ROU").alpha_2})
geo_dist = geo_dist.replace({"ROM": "ROU", "RO": pycountry.countries.get(alpha_3="ROU").alpha_2})

geo_info = geo_info.replace({"ZAR": "COD", "ZR": pycountry.countries.get(alpha_3="COD").alpha_2})
geo_dist = geo_dist.replace({"ZAR": "COD", "ZR": pycountry.countries.get(alpha_3="COD").alpha_2})

geo_info = geo_info.replace({"TMP": "TLS", "TP": pycountry.countries.get(alpha_3="TLS").alpha_2})
geo_dist = geo_dist.replace({"TMP": "TLS", "TP": pycountry.countries.get(alpha_3="TLS").alpha_2})

# drop Palestina data 
geo_info = geo_info[geo_info["iso3"] != "PAL"]
geo_dist = geo_dist[(geo_dist["iso_d"] != "PAL") & (geo_dist["iso_o"] != "PAL")]

# drop Channel Islands data
unesco_demo = unesco_demo[unesco_demo["iso3"] != "ZZA"]
unesco_education = unesco_education[unesco_education["iso3"] != "ZZA"]

In [11]:
geo_info = geo_info.append(geo_info[geo_info["iso3"] == "YUG"].replace({"YUG": "SRB", "YU": pycountry.countries.get(name="Serbia").alpha_2, "Serbia and Montenegro": "Serbia"}), ignore_index=True)
geo_dist = geo_dist.append(geo_dist[geo_dist["iso_o"] == "YUG"].replace({"YUG": "SRB", "YU": pycountry.countries.get(name="Serbia").alpha_2, "Serbia and Montenegro": "Serbia"}), ignore_index=True)
geo_dist = geo_dist.append(geo_dist[geo_dist["iso_d"] == "YUG"].replace({"YUG": "SRB", "YU": pycountry.countries.get(name="Serbia").alpha_2, "Serbia and Montenegro": "Serbia"}), ignore_index=True)

geo_info = geo_info.append(geo_info[geo_info["iso3"] == "YUG"].replace({"YUG": "MNE", "YU": pycountry.countries.get(name="Montenegro").alpha_2, "Serbia and Montenegro": "Montenegro"}), ignore_index=True)
geo_dist = geo_dist.append(geo_dist[geo_dist["iso_o"] == "YUG"].replace({"YUG": "MNE", "YU": pycountry.countries.get(name="Montenegro").alpha_2, "Serbia and Montenegro": "Montenegro"}), ignore_index=True)
geo_dist = geo_dist.append(geo_dist[geo_dist["iso_d"] == "YUG"].replace({"YUG": "MNE", "YU": pycountry.countries.get(name="Montenegro").alpha_2, "Serbia and Montenegro": "Montenegro"}), ignore_index=True)

# For the moment.. don't drop the YUG info
#geo_info = geo_info[(geo_info["iso3"] != "YUG")]
#geo_dist = geo_dist[(geo_dist["iso_d"] != "YUG") & (geo_dist["iso_o"] != "YUG")]

In [12]:
# overwrite the table files
geo_info.to_csv("Data_final/cepii_geo_info.csv", sep = "\t", index = False)
geo_dist.to_csv("Data_final/cepii_geo_dist.csv", sep = "\t", index = False)
unesco_demo.to_csv("Data_final/unesco_demo.csv", sep = "\t", index = False)
unesco_education.to_csv("Data_final/unesco_education.csv", sep = "\t", index = False)
resident_foreigners_norm.to_csv("Data_final/resident_foreigners_norm.csv", sep = "\t", index = False)

In [13]:
# dictionary with the structure: {iso2: iso3} --> we need iso2 not iso3 to translate the name country
iso2_all_countrie_engl = {pycountry.countries.get(alpha_3=i).alpha_2: i for i in iso3_all_countrie_engl.keys()}

In [14]:
# translate the countries using babel
# dictionary with the structure: {iso2: italian country name}
locale = Locale('it')
iso2_all_countrie_ita = {c: locale.territories[c] for c in iso2_all_countrie_engl.keys()}

In [15]:
# create a dataFrame with the structure iso2 | iso3 | english name | italian name | italian name istat |
iso_name = pd.DataFrame(columns=["iso2", "iso3", "english name", "italian name", "italian name istat"])
iso_name["iso2"] = sorted(list(iso2_all_countrie_engl.keys()))
iso_name["iso3"] = [iso2_all_countrie_engl[i] for i in iso_name["iso2"]]
iso_name["english name"] = [iso3_all_countrie_engl[i] for i in iso_name["iso3"]]
iso_name["italian name"] = [iso2_all_countrie_ita[i] for i in iso_name["iso2"]]
# add the "Serbia and Montenegro" row
iso_name = iso_name.append({"iso3": "YUG", "english name": "Serbia and Montenegro", "italian name": "Serbia e Montenegro"}, ignore_index = True)

In [16]:
# "found" is a dictionary with the structure: "offical-italian-name": "italian name (istat)"
not_found = []
found = {}
lower_names_ita = [i.lower() for i in list(set(iso_name["italian name"].values))]
lower_names_engl = [i.lower() for i in list(set(iso_name["english name"].values))]
for c in list(set(resident_foreigners_norm["Country"])):
    if c.lower() in lower_names_ita or c.lower() in lower_names_engl:
        try:
            temp_it = iso_name[iso_name["italian name"].str.lower() == c.lower()]["italian name"].values[0]
        except:
            temp_it = iso_name[iso_name["english name"].str.lower() == c.lower()]["italian name"].values[0]
        found[temp_it] = c
    else:
        not_found.append(c)
print(len(not_found), len(found))

30 168


In [17]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [18]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    
    return intersection_cardinality/float(union_cardinality)

In [19]:
# for all not assigned
my_stop = 0
stop = 3
not_match = {c: [] for c in not_found}
while my_stop < stop:
    my_stop += 1
    not_found2 = []
    for c in not_found:
        not_f = list(set(iso_name["italian name"]) - set(found.keys()) - set(not_match[c]))
       
        # find the most similar not yet assigne keys:
        temp = {n: similar(c, n) for n in not_f}
        question = (' --> ').join((c, max(temp.items(), key=operator.itemgetter(1))[0]))
        if yesno(question) == True:
            temp_it = iso_name[iso_name["italian name"] == max(temp.items(), key=operator.itemgetter(1))[0]]["italian name"].values[0]
            found[temp_it] = c
        else:
            not_match[c].append(max(temp.items(), key=operator.itemgetter(1))[0])
            not_f = list(set(iso_name["italian name"]) - set(found.keys()) - set(not_match[c]))
            temp = {n: jaccard_similarity(c, n) for n in not_f}
            question = (' --> ').join((c, max(temp.items(), key=operator.itemgetter(1))[0]))
            if yesno(question) == True:
                temp_it = iso_name[iso_name["italian name"] == max(temp.items(), key=operator.itemgetter(1))[0]]["italian name"].values[0]
                found[temp_it] = c
            else:
                not_match[c].append(max(temp.items(), key=operator.itemgetter(1))[0])
                not_found2.append(c)
                
    not_found = not_found2
               
print(len(not_found2), len(found))

Bosnia-Erzegovina --> Bosnia ed Erzegovina [Y/n] y
Sud Sudan, Repubblica del --> Repubblica Ceca [Y/n] n
Sud Sudan, Repubblica del --> Repubblica di Macedonia [Y/n] n
Congo, Repubblica democratica del (ex Zaire) --> Repubblica Centrafricana [Y/n] n
Congo, Repubblica democratica del (ex Zaire) --> Repubblica Dominicana [Y/n] n
Santa Sede (Stato della Città del Vaticano) --> Saint Pierre e Miquelon [Y/n] n
Santa Sede (Stato della Città del Vaticano) --> Sahara occidentale [Y/n] n
Zimbabwe (ex Rhodesia) --> Zimbabwe [Y/n] y
Riconosciuti non-cittadini (Lettonia) --> Repubblica di Macedonia [Y/n] n
Riconosciuti non-cittadini (Lettonia) --> Saint Lucia [Y/n] n
Serbia, Repubblica di --> Repubblica Ceca [Y/n] n
Serbia, Repubblica di --> Repubblica di Macedonia [Y/n] n
Micronesia, Stati Federati di --> Micronesia [Y/n] y
Salomone, Isole --> Isole Salomone [Y/n] y
Burkina Faso (ex Alto Volta) --> Burkina Faso [Y/n] y
Territori dell'Autonomia Palestinese --> Territori palestinesi [Y/n] y
Taiwan (

In [20]:
not_found

['Santa Sede (Stato della Città del Vaticano)',
 'Riconosciuti non-cittadini (Lettonia)',
 "Iran, Repubblica islamica dell'",
 'Kosovo',
 'Congo (Repubblica del)']

We have to manually add "Iran, Repubblica islamica dell'", "Congo (Repubblica del)" and "Congo, Repubblica democratica del (ex Zaire)"!

In [21]:
found["Congo-Brazzaville"] = "Congo (Repubblica del)"
#found["Congo-Kinshasa"] = "Congo, Repubblica democratica del (ex Zaire)"
found["Iran"] = "Iran, Repubblica islamica dell'"

In [22]:
# add the istat name (italian) column 
iso_name["italian name istat"] = iso_name["italian name"]
iso_name["italian name istat"] = iso_name["italian name istat"].replace(found)
# add the Serbia and Montenegro row
iso_name = iso_name.append({"iso2": "YU", "iso3": "YUG", "english name": "Serbia and Montenegro", 
                            "italian name": "Serbia e Montenegro", "italian name istat": "Serbia e Montenegro"},
                           ignore_index=True)

# Just take the information about countrie contained in the ISTAT db
#iso_name = iso_name[iso_name["italian name istat"] != "-"]
#iso_name.index  = range(len(iso_name))

In [23]:
iso_name.head()

,iso2,iso3,english name,italian name,italian name istat
0,AD,AND,Andorra,Andorra,Andorra
1,AE,ARE,United Arab Emirates,Emirati Arabi Uniti,Emirati Arabi Uniti
2,AF,AFG,Afghanistan,Afghanistan,Afghanistan
3,AG,ATG,Antigua and Barbuda,Antigua e Barbuda,Antigua e Barbuda
4,AI,AIA,Anguilla,Anguilla,Anguilla


In the ISTAT table before 2008 we have "Serbia and Montenegro" after we have "Serbia" and "Montenegro" as different countries. In the UNESCO tables we always have the two separeted. For coherence reasons, combine the two states before 2008 - combine only the absolute values (sum) and the values with a constant (over each year) normalization (e.g "current international $", but not "\% of GNI" or "\% of population" (it is not clear the population to consider, total, 15-24 and so on)).

Up to now, I only need the population --> let's just sum the two!

In [24]:
to_sum_idxs = ['200101', '200144', '200151', '200343', '200345']
#pop_to_weight_idxs = [ 'NY_GDP_DEFL_ZS', 'NY_GDP_MKTP_CD', 'NY_GDP_MKTP_CN',]

In [25]:
def sum_combination(idx, data):
    temp_y , temp_v = zip(*[(i, sum(j.Value)) for i, j in data[data["Indicator"] == idx].groupby(["Year"])])
    #data[data["Indicator"] == idx].groupby(["Year"])["Value"].transform(sum)
    result = []
    for y, v in zip(temp_y, temp_v):
        result.append({"Indicator": idx, "iso3": "YUG", "Year": y, "Value": v})
    #print(result)
    return(result)

In [26]:
# all the data about Serbia or Montenegro, before 2008
#a = unesco_demo.copy()
temp = unesco_demo[(unesco_demo["Year"] <= 2008) & ((unesco_demo["iso3"] ==  pycountry.countries.get(name="Serbia").alpha_3) | (unesco_demo["iso3"] ==  pycountry.countries.get(name="Montenegro").alpha_3))]
for i in to_sum_idxs:
    res = sum_combination(i, temp)
    for j in res:
        unesco_demo = unesco_demo.append(j, ignore_index=True)

The country not yet found are:
- 'Riconosciuti non-cittadini (Lettonia)',
- 'Santa Sede (Stato della Città del Vaticano)',
- 'Kosovo'

We actually don't care about 'Santa Sede (Stato della Città del Vaticano)', it should not really be considered as immigrant flow..

Problems: 
- 'Riconosciuti non-cittadini (Lettonia)': Non-citizens (Latvian: nepilsoņi) in Latvian law are individuals who are not citizens of Latvia or any other country but, who, in accordance with the Latvian law "Regarding the status of citizens of the former USSR who possess neither Latvian nor other citizenship", have the right to a non-citizen passport issued by the Latvian government as well as other specific rights. How do we handle it?
- 'Kosovo': is a disputed territory and partially recognised state in Southeastern Europe that declared independence from Serbia in February 2008 as the Republic of Kosovo.

For the moment. let's drop the not found countries from the resident foreigners table.

In [27]:
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Country"] != "Kosovo"]
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Country"] != "Santa Sede (Stato della Città del Vaticano)"]
resident_foreigners_norm = resident_foreigners_norm[resident_foreigners_norm["Country"] != "Riconosciuti non-cittadini (Lettonia)"]
resident_foreigners_norm.to_csv("Data_final/resident_foreigners_norm.csv", sep = "\t", index = False)
iso_name.to_csv("Data_final/country_name_coherence.csv", sep = "\t", index = False)
unesco_demo.to_csv("Data_final/unesco_demo.csv", sep = "\t", index = False)